In [7]:
# MOUNT YOUR GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')
# CHANGE CURRENT DIRECTORY TO FOLDER YOU HAvE THE CODE IN YOUR GOOGLE DRIVE
import os
# so that code from local project directory works here in Colab
os.chdir('/content/drive/MyDrive/TEAM_GREY/src')
os.getcwd()

'/Users/beanburger/Documents/_ML_UCL/_ML_MODULES/T2_MODULES/ASSIGNMENTS2/ADL_GROUP_CW/ADL_Team_Grey/augmented_data'

In [5]:
!pip install torch
!pip install torchvision
!pip install scikit-image
# conda install -c conda-forge scikit-image

In [10]:
import torch
import torchvision.transforms as T
from torchvision.utils import save_image
from torch.utils.data import DataLoader
import OxfordPetDataset as OxPetData  # Steve's `OxfordPetDataset` class from main file put into own file. I added in transform functionality to this class.
import config  # Contains Steve's `params` from the main file


ImportError: cannot import name 'v2' from 'torchvision.transforms' (/Users/beanburger/miniconda3/envs/comp0197-cw2-pt/lib/python3.12/site-packages/torchvision/transforms/__init__.py)

In [11]:
import torch
import torchvision.transforms as T
from torchvision.utils import save_image
from torch.utils.data import DataLoader
import OxfordPetDataset as OxPetData  # Steve's `OxfordPetDataset` class from main file put into own file. I added in transform functionality to this class.


ImportError: cannot import name 'v2' from 'torchvision.transforms' (/Users/beanburger/miniconda3/envs/comp0197-cw2-pt/lib/python3.12/site-packages/torchvision/transforms/__init__.py)

##### TRANSFORM WITHOUT AUGMENTATIONS (CALC MEAN & STD FOR NORMALISATION):

In [22]:
# expecting to receive a tensor to transform
transforms = T.Compose([
    T.Resize(size=(224, 224), antialias=True),
    # T.ToDtype(torch.float32, scale=True),  # Normalize expects float input
    # T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

script_dir = '../src/'
ox_dataset = OxPetData.OxfordPetDataset(image_dir=script_dir+'Oxford-IIIT-Pet/images',
                                        label_dir=script_dir+'Oxford-IIIT-Pet/annotations/trimaps',
                                        parameters=config.params, aug_transform=transforms)

ox_loader = DataLoader(dataset=ox_dataset, batch_size=64, shuffle=True)

def mean_std(loader):
    channels_sum, channels_sqd_sum, num_batches = 0, 0, 0

    for data, _ in loader:
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_sqd_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1

    mean_ = channels_sum/num_batches
    std_ = (channels_sqd_sum / num_batches - mean_ ** 2) ** 0.5
    return mean_, std_

TypeError: OxfordPetDataset.__init__() got an unexpected keyword argument 'aug_transform'

In [7]:
mean, std = mean_std(ox_loader)
print(mean)
print(std)
# a batch of 64 this gives:
# tensor([0.4812, 0.4493, 0.3957])
# tensor([0.2645, 0.2597, 0.2682])

# a batch of 128 this gives:
# tensor([0.4811, 0.4491, 0.3957])
# tensor([0.2643, 0.2594, 0.2679])

# a batch of 256 this gives:


tensor([0.4811, 0.4492, 0.3958])
tensor([0.2645, 0.2596, 0.2681])


##### TRANSFORM WITH AUGMENTATIONS (CALC MEAN & STD FOR NORMALISATION):

In [15]:
# expecting to receive a tensor to transform
aug_transform = T.Compose([
                     # T.ToPILImage(),
                     T.ColorJitter(brightness=0.5),
                     T.RandomHorizontalFlip(p=0.5),
                     T.RandomVerticalFlip(p=0.05),
                     T.RandomRotation(degrees=90),
                     T.RandomGrayscale(p=0.2),
                     T.ToDtype(torch.float32, scale=True),
                     # T.Normalize(mean=[0, 0, 0], std=[1, 1, 1])
                     ])

script_dir = '../src/'
aug_ox_dataset = OxPetData.OxfordPetDataset(image_dir=script_dir + 'Oxford-IIIT-Pet/images',
                                        label_dir=script_dir + 'Oxford-IIIT-Pet/annotations/trimaps',
                                        parameters=config.params, aug_transform=aug_transform)

aug_ox_loader = DataLoader(dataset=aug_ox_dataset, batch_size=256, shuffle=True)

NameError: name 'T' is not defined

In [14]:
mean, std = mean_std(aug_ox_loader)
print(mean)
print(std)

KeyboardInterrupt: 

##### APPLY THE NORMALISATION DETERMINED IN CELL ABOVE:

In [ ]:
aug_tfs = T.Compose([
                        # transforms.ToPILImage(),
                          T.ColorJitter(brightness=0.5),
                          T.RandomHorizontalFlip(p=0.5),
                          T.RandomVerticalFlip(p=0.05),
                          T.RandomRotation(degrees=90),
                          T.RandomGrayscale(p=0.2),
                          T.ToTensor(),
                          # T.Normalize(mean=[0, 0, 0], std=[1, 1, 1])
                          ])

script_dir = '../src/'
aug_ox_dataset = OxPetData.OxfordPetDataset(image_dir=script_dir + 'Oxford-IIIT-Pet/images',
                                        label_dir=script_dir + 'Oxford-IIIT-Pet/annotations/trimaps',
                                        parameters=config.params, transform=aug_tfs)

aug_ox_loader = DataLoader(dataset=aug_ox_dataset, batch_size=256, shuffle=True)

In [12]:
for img_num, (img, label) in enumerate(ox_dataset):
    if img_num == 5:
        break
    print(img.shape)
    save_image(img, 'img' + str(img_num))
    img_num += 1

torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])


##### CUTMIX & MIXUP AUGMENTATION:

In [10]:
from torchvision.transforms import v2

In [ ]:
script_dir = '../src/'
aug_ox_dataset = OxPetData.OxfordPetDataset(image_dir=script_dir + 'Oxford-IIIT-Pet/images',
                                        label_dir=script_dir + 'Oxford-IIIT-Pet/annotations/trimaps',
                                        parameters=config.params, transform=aug_tfs)

aug_ox_loader = DataLoader(dataset=aug_ox_dataset, batch_size=256, shuffle=True)